### **La Structure du Projet en 4 Grandes Phases**



#### **Phase 1 : La Préparation et le Remplissage de l'Entrepôt (ETL)**

C'est la phase où l'on collecte les matières premières et on les stocke proprement.

*   **Les Matières Premières :**
    *   Vos fichiers **CSV** contenant l'historique de production.
    *   Les données de l'**API Météo** (température, vent, etc.).

*   **Le "Plombier" (`DataHandler` et `CSVDataHandler`) :**
    *   **Mission :** Prendre les données brutes de chaque source (CSV, API), les nettoyer (`.clean()`), les fusionner, et les **INJECTER** dans la base de données.
    *   **Rôle :** Il est responsable de la qualité et de l'intégrité des données qui entrent dans votre système. C'est le gardien de l'entrepôt.

*   **L'"Architecte" (`Database`) :**
    *   **Mission :** Construire la structure de l'entrepôt (`.create_table()`). Il définit où chaque type de donnée sera stocké (table `Eolienne_data`, `Solaire_data`, etc.).
    *   **Rôle :** Il prépare le terrain pour que le "plombier" puisse travailler.

*   **Le Résultat de la Phase 1 :** Une base de données Supabase propre, bien structurée, et remplie de données historiques fiables, prêtes à être utilisées.

---

#### **Phase 2 : L'Entraînement du Modèle (Le Travail en Laboratoire)**

C'est la phase "intelligente", où l'on crée le "cerveau" de l'application. Ce processus est lancé manuellement par le développeur.

*   **Le "Chef d'Orchestre" (`run_training.py`) :**
    *   **Mission :** Orchestrer l'ensemble du processus d'entraînement pour un type d'énergie donné (ex: `python run_training.py eolienne`).
    *   **Rôle :** Il ne fait pas le travail lui-même, mais il appelle les bons experts dans le bon ordre.

*   **L'"Intendant" (`Producteur` et ses enfants `ProducteurEolien`, etc.) :**
    *   **Mission :** C'est l'expert qui sait exactement où trouver les bonnes données dans l'entrepôt (la BDD). Quand le chef d'orchestre lui demande les données pour l'éolien, il va chercher tout ce qu'il faut dans la table `Eolienne_data` et le fournit sous forme de DataFrame propre.
    *   **Rôle :** Il fait le pont entre le stockage technique (la BDD) et le besoin métier (données pour l'entraînement). Il est le **CONSOMMATEUR** des données préparées en Phase 1.

*   **Le "Scientifique" (`ModelTrainer`) :**
    *   **Mission :** Prendre les données fournies par l'intendant, appliquer des algorithmes complexes (`RandomForestRegressor`), trouver le meilleur modèle possible, et le sauvegarder précieusement.
    *   **Rôle :** C'est le cerveau qui crée la logique de prédiction.

*   **Le Résultat de la Phase 2 :** Des fichiers de modèle (`.pkl`) sauvegardés sur le disque. Chaque fichier est un "cerveau" spécialisé, entraîné pour prédire la production d'un type d'énergie.

---

#### **Phase 3 : L'Exposition du Modèle (L'API)**

Maintenant que le cerveau est prêt, il faut le brancher pour qu'on puisse lui poser des questions.

*   **Le "Serveur Vocal" (`api/main.py` avec FastAPI) :**
    *   **Mission :** Exposer le modèle au monde extérieur via des URLs (des "routes" ou "endpoints") comme `/predict/solar`.
    *   **Rôle :** Il attend les appels, charge le bon "cerveau" (`.pkl`) grâce à la méthode `.load()` de la classe `ModelTrain`, lui transmet les nouvelles données (prévisions météo), et renvoie la réponse. Il ne fait aucun entraînement.

*   **Le Résultat de la Phase 3 :** Une API fonctionnelle, prête à recevoir des demandes de prédiction et à y répondre en temps réel.

---

#### **Phase 4 : L'Interaction avec l'Utilisateur (L'Interface)**

C'est la partie visible de l'iceberg, celle que l'utilisateur final voit et utilise.

*   **L'"Interface Utilisateur" (`streamlit_app.py`) :**
    *   **Mission :** Offrir une expérience simple à l'utilisateur.
    *   **Rôle :**
        1.  Pour **afficher l'historique**, elle demande directement à l'expert (`Producteur`) de lui fournir les données pour une période donnée (`.get_production_history()`).
        2.  Pour **faire une prédiction**, elle contacte le "serveur vocal" (l'API FastAPI), lui envoie la demande, et affiche joliment la réponse à l'utilisateur.

*   **Le Résultat de la Phase 4 :** Une application web fonctionnelle où un utilisateur peut visualiser des données passées et obtenir des prédictions sur la production future.

### **Schéma Simplifié du Flux**

**Phase 1 & 2 (Développement / Off-line) :**
`CSV/API` -> `DataHandler` -> `Database` <- `Producteur` -> `ModelTrainer` -> `Fichier .pkl`

**Phase 3 & 4 (Production / On-line) :**
`Utilisateur` -> `Streamlit` -> `FastAPI` -> `ModelTrainer.load()` -> `Prédiction`